# ASSOCIATION ANALYSIS WITH ECLAT

**File:** Eclat.ipynb

**Course:** Data Science Foundations: Data Mining in Python

# INSTALL AND IMPORT LIBRARIES
The Python library `pyECLAT` contains the implementation of the Eclat algorithm, which can be installed with Python's `pip` command. This command only needs to be done once per machine.

The standard, shorter approach may work:

In [ ]:
# pip install pyECLAT

If the above command didn't work, it may be necessary to be more explicit, in which case you could run the code below.

In [ ]:
# import sys
# !{sys.executable} -m pip install pyECLAT

Once `pyECLAT` is installed, then load the libraries below.

In [ ]:
import pandas as pd              # For dataframes
import matplotlib.pyplot as plt  # For plotting data
from pyECLAT import ECLAT        # For Eclat algorithm

# LOAD AND PREPARE DATA
For this demonstration, we'll use the dataset `Groceries.csv`, which comes from the R package `arules` and is saved as a CSV file. The data is in transactional format (as opposed to tabular format), which means that each row is a list of items purchased together and that the items may be in different order. There are 32 columns in each row, each column either contains a purchased items or NaN.

## Import Data

- To read read the dataset from a local CSV file, run the following cell.

In [ ]:
df = pd.read_csv('data/Groceries.csv', header=None)

df.head()

# APPLY ECLAT

Call `ECLAT()` on `transactions_df` to fit a model. For parameters, `ECLAT.fit()` can take the minimum support, minimum confidence, minimum lift, and minimum items in a transaction. Only the pairs of items that satisfy these criteria will be returned.

As a note, increasing the number for `max_combination` will dramatically increase the processing time. With a value of 2, processing takes a few seconds; with a value of 3, processing takes minutes.

In [ ]:
eclat_instance = ECLAT(data=df, verbose=True)

_, supports = eclat_instance.fit(
    min_support=0.03,
    min_combination=2,
    max_combination=2,
    separator=' & ',
    verbose=True)

In [ ]:
supports

## Convert Rules to Readable Format

This code takes the rules above and formats them in a table format that is easier to read and modify.

In [ ]:
rows = []

for rule, support in supports.items():
    from_, to = rule.split(' & ')
    rows.append({
        'From': from_,
        'To': to,
        'Support': support
    })
    
rules_df = pd.DataFrame(rows)

rules_df.head()

## List Rules with N's
The code below calls `plot()` on each row of the rules DataFrame to create a list of all the mined rules. First, we have to add two numeric columns corresponding to each item to `rules_df`.

In [ ]:
# List of all items
items = set(rules_df['From']) | set(rules_df['To'])

# Creates a mapping of items to numbers
imap = {item : i for i, item in enumerate(items)}

# Maps the items to numbers and adds the numeric 'FromN' and 'ToN' columns
rules_df['FromN'] = rules_df['From'].map(imap)
rules_df['ToN'] = rules_df['To'].map(imap)

# Displays the top 20 association rules, sorted by Support
rules_df.head(20).sort_values(by='Support', ascending=False)

## Plot Rules
Plot each pair of items in the rule. If a rule is A->B, then the item A is in the bottom row of the plot (y=0) and B is in the top row (y=1). The color of each line indicates the support of the rule multiplied by 100 (support*100). The width of each line is controlled by the confidence of each rule.

In [ ]:
# Adds ticks to the top of the graph also
plt.rcParams['xtick.top'] = plt.rcParams['xtick.labeltop'] = True

# Sets the size of the plot
fig = plt.figure(figsize=(15, 7))

# Draws a line between items for each rule
# Colors each line according to the support of the rule
for index, row in rules_df.iterrows():
    plt.plot([row['FromN'], row['ToN']], [0, 1], 'o-',
             c=plt.cm.viridis(row['Support'] * 10),
             markersize=20)

# Adds a colorbar and its title  
cb = plt.colorbar(plt.cm.ScalarMappable(cmap='viridis'))
cb.set_label('Support*10')

# Adds labels to xticks and removes yticks
plt.xticks(range(len(items)), items, rotation='vertical')
plt.yticks([])
plt.show()

# CLEAN UP

- If desired, clear the results with Cell > All Output > Clear. 
- Save your work by selecting File > Save and Checkpoint.
- Shut down the Python kernel and close the file by selecting File > Close and Halt.